# 数理最適化

## モデリングと定式化

### 生産計画問題 - 線形最適化 -
**2製品の生産計画**
- 合金A,Bの生産時間を決定したい→ 変数
- 利益を最大化したい → 目的関数
- 生産時間の合計は40時間以下 → 制約
- 各合金の生産上限を満たす → 制約

合金|生産効率<br />[トン/時間]|利益<br />[万円/トン]|生産上限<br />[トン]
:--|--:|--:|--:
A|2|9|50
B|1|11|25

In [ ]:
%matplotlib inline
import numpy as np, pandas as pd, matplotlib.pyplot as plt, networkx as nx
from collections import OrderedDict
from pulp import *
from ortoolpy import addvar, addvars
from answer import answer

In [ ]:
a = pd.DataFrame(OrderedDict([
        ('合金',     ['A', 'B']),
        ('生産効率', [2, 1]),  # [トン/時間]
        ('利益',     [9, 11]), # [万円/トン]
        ('生産上限', [50, 25]),# [トン]
        ('Var', addvars(2)),   # [時間]
    ]))
a

In [ ]:
m = LpProblem(sense=LpMaximize)
m += lpDot(a.生産効率 * a.利益, a.Var)
m += lpSum(a.Var) <= 40
for _, r in a.iterrows():
    m += r.生産効率 * r.Var <= r.生産上限
m.solve()
a['Val'] = a.Var.apply(value)
a.Val

### 輸送問題 - ネットワーク型LP -
**ソナパニック者の輸送計画**

- 倉庫群から工場群への輸送量を決めたい → 変数
- 輸送コストを最小化したい → 目的関数
- 倉庫からの搬出は、供給可能量以下 → 制約
- 工場への搬入は、需要量以上 → 制約

<table>
<tr><td rowspan="2" colspan="2">輸送費</td><td colspan="4">組み立て工場</td></tr>
<tr><td>T1</td><td>T2</td><td>T3</td><td>T4</td><td>供給</td></tr>
<tr><td rowspan="3">倉庫</td><td>A</td><td>40</td><td>48</td><td>21</td><td>15</td><td>25</td></tr>
<tr><td>B</td><td>52</td><td>35</td><td>45</td><td>60</td><td>35</td></tr>
<tr><td>C</td><td>25</td><td>43</td><td>70</td><td>85</td><td>40</td></tr>
<tr><td></td><td>需要</td><td>15</td><td>20</td><td>35</td><td>30</td></tr>
</table>



In [ ]:
供給 = {'A': 25, 'B': 35, 'C':40}
需要 = {'T1': 15, 'T2': 20, 'T3': 35, 'T4': 30}
a = pd.DataFrame([(i, j) for i in ['A', 'B', 'C']
                 for j in ['T1', 'T2', 'T3', 'T4']], columns=['倉庫', '工場'])
a['輸送費'] = [40,48,21,15,52,35,45,60,25,43,70,85]
a['Var'] = addvars(len(a))
a

In [ ]:
answer('1.1.2')

### 研究室配属問題

- 25人の学生が5つの研究室への希望度(0-100)を出している
- 各研究室に5人ずつ配属される
- 配属された学生の希望度の総和を最大化せよ

In [ ]:
nn = [[int(i) for i in s.split(',')] for s in """\
0,100,80,0,50
100,100,0,0,0
0,0,100,0,0
40,50,30,80,100
40,0,100,100,0
0,0,60,100,70
0,100,100,0,0
30,20,59,100,50
10,10,100,60,60
0,0,100,100,0
0,0,0,100,100
20,50,100,0,0
100,0,0,0,0
40,30,30,30,100
25,0,70,70,100
0,0,100,100,100
100,100,100,100,100
0,0,100,0,0
10,30,80,80,100
0,0,100,100,0
0,0,60,100,100
0,0,0,50,100
0,100,0,100,0
0,0,90,90,100
20,20,20,100,20""".split('\n')]
a = pd.DataFrame([(i, j, k) for i, n in enumerate(nn)
    for j, k in enumerate(n)], columns=['学生', '研究室', '希望度'])
n学生 = len(nn)
a[:3]

In [ ]:
answer('1.1.3')

### ナップサック問題 - 0-1整数最適化 -
**海梨市のプロジェクト**

- 費用が2000万円以下で、便益の総和が最大になるように企画を選択せよ

企画|A|B|C|D|E|F|G|H|I|J
:--|--:|--:|--:|--:|--:|--:|--:|--:|--:|--:
便益|4|5|3|6|13|23|11|7|15|9
費用|3|4|2|5|10|15|6|4|13|7

In [ ]:
a = pd.DataFrame({
        '便益': [4, 5, 3, 6, 13, 23, 11, 7, 15, 9],
        '費用': [3, 4, 2, 5, 10, 15, 6, 4, 13, 7]
    })
a[:3]

In [ ]:
answer('1.1.4')

### おせち受注問題 - 混合整数最適化 -
- 3つの店舗に合計15個のおせちを発注したい
- ある店舗に1つ以上発注する場合、固定費用がかかる
- 利益を最大にするには、各々何個ずつ発注すればよいか

|A|B|C
:--|--:|--:|--:
固定費用|10|50|6
1個当たり利益|6.5|18|4
上限|8|6|20

In [ ]:
a = pd.DataFrame(OrderedDict([
        ('固定費用', [10, 50, 6]),
        ('利益', [6.5, 18, 4]),
        ('上限', [8, 6, 20]),
    ]), index=['A', 'B', 'C'])
a

In [ ]:
answer('1.1.5')

### おせち問題再考
- 最大と最小の差が2以下になるようにせよ

In [ ]:
answer('1.1.6')